In [1]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import confusion_matrix

In [2]:
# Leemos los datos y los cargamos a un DataFrame de pandas
data = pd.read_csv('breast_cancer.csv',header=None)
data.columns = ['Sample Code Number','Clump Thickness','Uniformity of Cell Size',
                'Uniformity of Cell Shape','Marginal Adhesion','Single Epithelial Cell Size',
                'Bare Nuclei','Bland Chromatin','Normal Nucleoli','Mitoses','Class']
data.head(5)

,Sample Code Number,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


In [3]:
# Aplicamos un poco de pre-procesamiento a los datos
data = data.drop(['Sample Code Number'],axis=1) #Eliminamos la 1era columna
data = data[data['Bare Nuclei'] != '?'] #Removemos filas sin datos
data['Class'] = np.where(data['Class'] == 2,0,1) #Cambiamos la representacion de las clases
data['Class'].value_counts() #Distribucion de las slases

0    444
1    239
Name: Class, dtype: int64

In [4]:
# Separamos features de target
X = data.drop(['Class'],axis=1)
y = data['Class']

# Dividimos los datos en Train y Test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [5]:
# Definimos el modelo y sus parámetros
knn = KNeighborsClassifier()

# Grilla para Grid Search
param_grid = {'n_neighbors':np.arange(1, 20),
              'leaf_size':[1,3,5,7,10],
              'algorithm':['auto', 'kd_tree']}

# Grilla para Random Search
param_dist = {'n_neighbors':sp.stats.randint(1, 20),
              'leaf_size':sp.stats.randint(1, 10),
              'algorithm':['auto', 'kd_tree']}

In [14]:
# CASO 1: Grid Search
#model = GridSearchCV(knn, param_grid=param_grid, cv=5)

# CASO 2: Random Search
model = RandomizedSearchCV(knn, param_dist, n_iter=190, random_state=0, cv=5)

# Entrenamos: KNN con la grilla definida arriba y CV con tamaño de Fold=5
model.fit(X_train, y_train)

RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=KNeighborsClassifier(algorithm='auto',
                                                  leaf_size=30,
                                                  metric='minkowski',
                                                  metric_params=None,
                                                  n_jobs=None, n_neighbors=5,
                                                  p=2, weights='uniform'),
                   iid='warn', n_iter=190, n_jobs=None,
                   param_distributions={'algorithm': ['auto', 'kd_tree'],
                                        'leaf_size': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fea5d3340f0>,
                                        'n_neighbors': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fea5d36a9e8>},
                   pre_dispatch='2*n_jobs', random_state=0, refit=True,
                   return_train_score=False, scoring=None, ver

In [15]:
print("Mejores parametros: "+str(model.best_params_))
print("Mejor Score: "+str(model.best_score_)+'\n')

scores = pd.DataFrame(model.cv_results_)
scores

Mejores parametros: {'algorithm': 'kd_tree', 'leaf_size': 8, 'n_neighbors': 15}
Mejor Score: 0.97265625



,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_algorithm,param_leaf_size,param_n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.003447,0.001000,0.006961,0.001725,auto,6,1,"{'algorithm': 'auto', 'leaf_size': 6, 'n_neigh...",0.971154,0.950980,0.960784,0.960784,0.931373,0.955078,0.013445,166
1,0.002264,0.000164,0.004943,0.000340,kd_tree,4,8,"{'algorithm': 'kd_tree', 'leaf_size': 4, 'n_ne...",0.980769,0.950980,0.960784,0.990196,0.941176,0.964844,0.018241,99
2,0.002002,0.000026,0.004943,0.000127,kd_tree,4,19,"{'algorithm': 'kd_tree', 'leaf_size': 4, 'n_ne...",0.980769,0.960784,0.970588,0.990196,0.950980,0.970703,0.013906,25
3,0.002744,0.001530,0.004910,0.000818,auto,8,7,"{'algorithm': 'auto', 'leaf_size': 8, 'n_neigh...",0.980769,0.950980,0.960784,0.990196,0.941176,0.964844,0.018241,99
4,0.002345,0.000264,0.004933,0.000536,auto,9,13,"{'algorithm': 'auto', 'leaf_size': 9, 'n_neigh...",0.980769,0.960784,0.970588,0.990196,0.950980,0.970703,0.013906,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,0.001997,0.000004,0.004783,0.000051,kd_tree,4,15,"{'algorithm': 'kd_tree', 'leaf_size': 4, 'n_ne...",0.980769,0.960784,0.980392,0.990196,0.950980,0.972656,0.014431,1
186,0.001991,0.000005,0.004510,0.000025,kd_tree,4,7,"{'algorithm': 'kd_tree', 'leaf_size': 4, 'n_ne...",0.980769,0.950980,0.960784,0.990196,0.941176,0.964844,0.018241,99
187,0.001970,0.000013,0.004233,0.000016,auto,7,7,"{'algorithm': 'auto', 'leaf_size': 7, 'n_neigh...",0.980769,0.950980,0.960784,0.990196,0.941176,0.964844,0.018241,99
188,0.001972,0.000007,0.004194,0.000035,auto,7,6,"{'algorithm': 'auto', 'leaf_size': 7, 'n_neigh...",0.971154,0.950980,0.960784,0.990196,0.941176,0.962891,0.016896,160


In [16]:
#Predecimos en los datos de test
prediction = model.predict(X_test)

In [17]:
# Matriz de Confusion
cm = confusion_matrix(y_test,prediction)
print(cm)

# Reporte de Clasificacion
report = classification_report(y_test, prediction)
print(report)

[[102   1]
 [  8  60]]
              precision    recall  f1-score   support

           0       0.93      0.99      0.96       103
           1       0.98      0.88      0.93        68

    accuracy                           0.95       171
   macro avg       0.96      0.94      0.94       171
weighted avg       0.95      0.95      0.95       171

